In [2]:
import six
from os.path import isfile,expanduser
import numpy as np
import h5py
import illustris_python as il
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy as sp
import os
import pickle

In [6]:
basepaths = {'TNG100': ['/virgotng/universe/IllustrisTNG/TNG100-1/postprocessing/XrayEmission/',99],
             'SIMBA': ['/virgotng/universe/Simba/L100n1024FP/postprocessing/XrayEmission/',151],
             'EAGLE': ['/virgotng/universe/Eagle/Eagle100-1/postprocessing/XrayEmission/',28]  }


basepaths2 = {'TNG100': ['/virgotng/universe/IllustrisTNG/TNG100-1/output',99],
              'SIMBA': ['/virgotng/universe/Simba/L100n1024FP/output',151],
              'EAGLE': ['/virgotng/universe/Eagle/Eagle100-1/output',28]  }

In [3]:
def shift(u, v, box_length):
    """
    returns the position vector u-v in a periodic Cartesian box
    
    Parameters
    ----------
    u : N x 3 position array
    v : N x 3 OR 1 x 3 position array
    box_length : float in same units as [u], [v]
    
    Returns
    -------
    result: N x M position vector (array)
    """

    result = u - v
    result[result > box_length / 2.0] -= box_length
    result[result < -box_length / 2.0] += box_length
    
    return result



def mass_mask(basepath, snapshot, mi = 10**12.5 , ma = 10**14.3):
    '''
    takes the basepath and snapshot number as inputs for a sim
    and applies a mask in Group_M_Crit200
    returns the masses and the ids of these groups
    '''
    
    # Loading header
    header = il.groupcat.loadHeader(basepath, snapshot)
    h = header['HubbleParam']
    a = header['Time']

    # Loading halo data
    GroupMass = il.groupcat.loadHalos(basepath, snapshot, fields=['Group_M_Crit200']) * 10**10 / h  # Msun

    # Apply mass cut
    haloIDs = np.where((GroupMass > mi) & (GroupMass < ma))[0]
    GroupMass = GroupMass[haloIDs]
        
    return GroupMass, haloIDs

In [4]:
def getNumPart(header):
    """ Calculate number of particles of all types given a snapshot header. """
    if 'NumPart_Total_HighWord' not in header:
        return header['NumPart_Total'] # new uint64 convention

    nTypes = 6

    nPart = np.zeros(nTypes, dtype=np.int64)
    for j in range(nTypes):
        nPart[j] = header['NumPart_Total'][j] | (header['NumPart_Total_HighWord'][j] << 32)

    return nPart



def partTypeNum(partType):
    """ Mapping between common names and numeric particle types. """
    if str(partType).isdigit():
        return int(partType)
        
    if str(partType).lower() in ['gas','cells']:
        return 0
    if str(partType).lower() in ['dm','darkmatter']:
        return 1
    if str(partType).lower() in ['dmlowres']:
        return 2 # only zoom simulations, not present in full periodic boxes
    if str(partType).lower() in ['tracer','tracers','tracermc','trmc']:
        return 3
    if str(partType).lower() in ['star','stars','stellar']:
        return 4 # only those with GFM_StellarFormationTime>0
    if str(partType).lower() in ['wind']:
        return 4 # only those with GFM_StellarFormationTime<0
    if str(partType).lower() in ['bh','bhs','blackhole','blackholes']:
        return 5
    
    raise Exception("Unknown particle type name.")
    
    
    
def gcPath(basePath, snapNum, chunkNum=0):
    """ Return absolute path to a group catalog HDF5 file (modify as needed). """
    gcPath = basePath + '../../output/groups_%03d/' % snapNum
    filePath1 = gcPath + 'groups_%03d.%d.hdf5' % (snapNum, chunkNum)
    filePath2 = gcPath + 'fof_subhalo_tab_%03d.%d.hdf5' % (snapNum, chunkNum)

    if isfile(expanduser(filePath1)):
        return filePath1
    return filePath2



def offsetPath(basePath, snapNum):
    """ Return absolute path to a separate offset file (modify as needed). """
    offsetPath = basePath + '/../offsets/offsets_%03d.hdf5' % snapNum

    return offsetPath



def getSnapOffsets(basePath, snapNum, id, type):
    """ Compute offsets within snapshot for a particular group/subgroup. """
    r = {}

    # old or new format
    if 'fof_subhalo' in gcPath(basePath, snapNum):
        # use separate 'offsets_nnn.hdf5' files
        with h5py.File(offsetPath(basePath, snapNum), 'r') as f:
            groupFileOffsets = f['FileOffsets/'+type][()]
            r['snapOffsets'] = np.transpose(f['FileOffsets/SnapByType'][()])  # consistency
    else:
        # load groupcat chunk offsets from header of first file
        with h5py.File(gcPath(basePath, snapNum), 'r') as f:
            groupFileOffsets = f['Header'].attrs['FileOffsets_'+type]
            r['snapOffsets'] = f['Header'].attrs['FileOffsets_Snap']

    # calculate target groups file chunk which contains this id
    groupFileOffsets = int(id) - groupFileOffsets
    fileNum = np.max(np.where(groupFileOffsets >= 0))
    groupOffset = groupFileOffsets[fileNum]

    # load the length (by type) of this group/subgroup from the group catalog
    with h5py.File(gcPath(basePath, snapNum, fileNum), 'r') as f:
        r['lenType'] = f[type][type+'LenType'][groupOffset, :]

    # old or new format: load the offset (by type) of this group/subgroup within the snapshot
    if 'fof_subhalo' in gcPath(basePath, snapNum):
        with h5py.File(offsetPath(basePath, snapNum), 'r') as f:
            r['offsetType'] = f[type+'/SnapByType'][id, :]

            # add TNG-Cluster specific offsets if present
            if 'OriginalZooms' in f:
                for key in f['OriginalZooms']:
                    r[key] = f['OriginalZooms'][key][()] 
    else:
        with h5py.File(gcPath(basePath, snapNum, fileNum), 'r') as f:
            r['offsetType'] = f['Offsets'][type+'_SnapByType'][groupOffset, :]

    return r



def loadSubset(basePath, snapNum, partType, fields=None, subset=None, mdi=None, sq=True, float32=False):
    """ Load a subset of fields for all particles/cells of a given partType.
        If offset and length specified, load only that subset of the partType.
        If mdi is specified, must be a list of integers of the same length as fields,
        giving for each field the multi-dimensional index (on the second dimension) to load.
          For example, fields=['Coordinates', 'Masses'] and mdi=[1, None] returns a 1D array
          of y-Coordinates only, together with Masses.
        If sq is True, return a numpy array instead of a dict if len(fields)==1.
        If float32 is True, load any float64 datatype arrays directly as float32 (save memory). """
    result = {}

    ptNum = partTypeNum(partType)
    gName = "PartType" + str(ptNum)

    # make sure fields is not a single element
    if isinstance(fields, six.string_types):
        fields = [fields]

    # load header from first chunk
    with h5py.File(il.snapshot.snapPath(basePath, snapNum), 'r') as f:

        header = dict(f['Header'].attrs.items())
        nPart = il.snapshot.getNumPart(header)

        # decide global read size, starting file chunk, and starting file chunk offset
        if subset:
            offsetsThisType = subset['offsetType'][ptNum] - subset['snapOffsets'][ptNum, :]

            fileNum = np.max(np.where(offsetsThisType >= 0))
            fileOff = offsetsThisType[fileNum]
            numToRead = subset['lenType'][ptNum]
        else:
            fileNum = 0
            fileOff = 0
            numToRead = nPart[ptNum]

        result['count'] = numToRead

        if not numToRead:
            # print('warning: no particles of requested type, empty return.')
            return result

        # find a chunk with this particle type
        i = 1
        while gName not in f:
            f = h5py.File(il.snapshot.snapPath(basePath, snapNum, i), 'r')
            i += 1

        # if fields not specified, load everything
        if not fields:
            fields = list(f[gName].keys())

        for i, field in enumerate(fields):
            # verify existence
            if field not in f[gName].keys():
                raise Exception("Particle type ["+str(ptNum)+"] does not have field ["+field+"]")

            # replace local length with global
            shape = list(f[gName][field].shape)
            shape[0] = numToRead

            # multi-dimensional index slice load
            if mdi is not None and mdi[i] is not None:
                if len(shape) != 2:
                    raise Exception("Read error: mdi requested on non-2D field ["+field+"]")
                shape = [shape[0]]

            # allocate within return dict
            dtype = f[gName][field].dtype
            if dtype == np.float64 and float32: dtype = np.float32
            result[field] = np.zeros(shape, dtype=dtype)

    # loop over chunks
    wOffset = 0
    origNumToRead = numToRead

    while numToRead:
        f = h5py.File(il.snapshot.snapPath(basePath, snapNum, fileNum), 'r')

        # no particles of requested type in this file chunk?
        if gName not in f:
            f.close()
            fileNum += 1
            fileOff  = 0
            continue

        # set local read length for this file chunk, truncate to be within the local size
        numTypeLocal = f['Header'].attrs['NumPart_ThisFile'][ptNum]

        numToReadLocal = numToRead

        if fileOff + numToReadLocal > numTypeLocal:
            numToReadLocal = numTypeLocal - fileOff

        #print('['+str(fileNum).rjust(3)+'] off='+str(fileOff)+' read ['+str(numToReadLocal)+\
        #      '] of ['+str(numTypeLocal)+'] remaining = '+str(numToRead-numToReadLocal))

        # loop over each requested field for this particle type
        for i, field in enumerate(fields):
            # read data local to the current file
            if mdi is None or mdi[i] is None:
                result[field][wOffset:wOffset+numToReadLocal] = f[gName][field][fileOff:fileOff+numToReadLocal]
            else:
                result[field][wOffset:wOffset+numToReadLocal] = f[gName][field][fileOff:fileOff+numToReadLocal, mdi[i]]

        wOffset   += numToReadLocal
        numToRead -= numToReadLocal
        fileNum   += 1
        fileOff    = 0  # start at beginning of all file chunks other than the first

        f.close()

    # verify we read the correct number
    if origNumToRead != wOffset:
        raise Exception("Read ["+str(wOffset)+"] particles, but was expecting ["+str(origNumToRead)+"]")

    # only a single field? then return the array instead of a single item dict
    if sq and len(fields) == 1:
        return result[fields[0]]

    return result



def loadHalo(basePath, snapNum, id, partType, fields=None):
    """ Load all particles/cells of one type for a specific halo
        (optionally restricted to a subset fields). """
    
    # load halo length, compute offset, call loadSubset
    subset = getSnapOffsets(basePath, snapNum, id, "Group")
    
    return loadSubset(basePath, snapNum, partType, fields, subset=subset)

In [5]:
def Xray_dict(sim_dict, sim_dict2):
    '''
    takes 2 dicts as input, sim_dict is { sim: [basepath,snapshot] } where basepath locates the Xray data
    sim_dict2 is the same where basepath2 locates the output data (regular path)
    returns a nested dict { sim: { haloID: { Xray_emission: values(for each gas cell) } } }
    '''
    
    # initialise output dict
    result = {}
    
    #loop for each sim
    for sim2, basepath2 in sim_dict2.items():
        
        # get the ids of relevant mass halos
        group_masses, haloIDs = mass_mask(basepath2[0], basepath2[1]) 
        
        
        #loop for each sim
        for sim, basepath in sim_dict.items() :
            if sim == sim2:
                # start a new dict to save values of each sim
                result[sim] = {}


                #loop over the halo ids
                for index, haloID in enumerate(haloIDs):

                    halo_key = f'{haloID}'
                    result[sim][halo_key] = {}

                    Xray_em = loadHalo(basepath[0], basepath[1], haloID , 0, fields = ['Xray_Emission_05_2keV', 'Xray_Emission_03_2keV']) # erg*cm³/s

                    result[sim][halo_key]['Xray_Emission_05_2keV'] = Xray_em['Xray_Emission_05_2keV'] # erg*cm³/s
                    result[sim][halo_key]['Xray_Emission_03_2keV'] = Xray_em['Xray_Emission_03_2keV'] # erg*cm³/s

    return result

In [7]:
# Save the dictionary 
#with open('../data/xray_em.pickle', 'wb') as file:
#    pickle.dump(Xray_dict(basepaths, basepaths2), file)


In [8]:
def Xray_lumdict(sim_dict, sim_dict2):
    '''
    takes 2 dicts as input, sim_dict is { sim: [basepath, snapshot] } where basepath locates the Xray data
    sim_dict2 is the same where basepath2 locates the output data (regular path)
    returns a nested dict { sim: { haloID: { field : value array } } }
    '''
    
    # initialise output dict
    result = {}
    
    #loop for each sim
    for sim2, basepath2 in sim_dict2.items():
        
        # load header
        header = il.groupcat.loadHeader(basepath2[0] , basepath2[1] )
        h = header['HubbleParam']
        a = header['Time']
        BoxSize = header['BoxSize'] *a/h # pkpc
        
        # get the ids of relevant mass halos
        group_masses, haloIDs = mass_mask(basepath2[0], basepath2[1]) # Msun
        
        # load all halos
        Halos = il.groupcat.loadHalos(basepath2[0], basepath2[1], fields=['GroupPos', 'Group_R_Crit500', 'Group_R_Crit200'])
        
        #loop for each sim
        for sim, basepath in sim_dict.items() :
            if sim == sim2:
                
                # start a new dict to save values of each sim
                result[sim] = {}
                
                #loop over the halo ids
                for index, haloID in enumerate(haloIDs):

                    halo_key = f'{haloID}'
                    result[sim][halo_key] = {}
                    
                    # define halo data
                    halo_pos = Halos['GroupPos'][haloID] *a/h  # pkpc
                    R200c = Halos['Group_R_Crit200'][haloID] *a/h # pkpc
                    R500c = Halos['Group_R_Crit500'][haloID] *a/h # pkpc
                    
                    # load gas data
                    Gas = il.snapshot.loadHalo(basepath2[0] , basepath2[1] , haloID, 0)            
                    Gas_mass = Gas['Masses'] *10**10/h  # Msun
                    Gas_ea = Gas['ElectronAbundance'] # unitless
                    Gas_density = Gas['Density'] *(10**10/h)/(a/h)**3 # Msun/pkpc³
                    Gas_coords = Gas['Coordinates'] *a/h  # pkpc
                    Centred_gas = shift(Gas_coords, halo_pos, BoxSize) # pkpc
                    Gas_radius = np.linalg.norm(Centred_gas, axis =1) # pkpc
                    
                    # load xray data
                    Xray_em = loadHalo(basepath[0], basepath[1], haloID, 0, fields = ['Xray_Emission_05_2keV', 'Xray_Emission_03_2keV']) # erg*cm³/s
                    
                    # calc Lx
                    nh_conv = 6.768 *10**-32 # (Msun/pkpc**3 to g/cm**3)
                    Xray_05 = Xray_em['Xray_Emission_05_2keV']  # erg * cm**3 * s*-1
                    n_h = Gas_density * (0.76/1.6726e-24) * nh_conv  # cm**-3
                    n_e = Gas_ea * n_h # cm**-3
                    Vol = Gas_mass/Gas_density *(3.086*10**21)**3 # cm**3
                    Xray_lum =  Xray_05 * n_e * n_h * Vol # erg/s
                    
                    result[sim][halo_key]['Lx_05_2keV'] = Xray_lum # erg*s
                    result[sim][halo_key]['radius'] = Gas_radius # pkpc
                    result[sim][halo_key]['R200c'] = R200c # pkpc
                    result[sim][halo_key]['R500c'] = R500c # pkpc
                    
                   
    return result

In [9]:
#with open('../data/xray_lum.pickle', 'wb') as file:
#    pickle.dump(Xray_lumdict(basepaths, basepaths2), file)


In [10]:
def Xray_line_dict(sim_dict, sim_dict2):
    '''
    takes 2 dicts as input, sim_dict is { sim: [basepath,snapshot] } where basepath locates the Xray data
    sim_dict2 is the same where basepath2 locates the output data (regular path)
    returns a nested dict { sim: { haloID: { Xray_emission: values(for each gas cell) } } }
    '''
    
    # initialise output dict
    result = {}
    
    #loop for each sim
    for sim2, basepath2 in sim_dict2.items():
        
        # get the ids of relevant mass halos
        group_masses, haloIDs = mass_mask(basepath2[0], basepath2[1]) 
        
        
        #loop for each sim
        for sim, basepath in sim_dict.items() :
            if sim == sim2:
                # start a new dict to save values of each sim
                result[sim] = {}


                #loop over the halo ids
                for index, haloID in enumerate(haloIDs):

                    halo_key = f'{haloID}'
                    result[sim][halo_key] = {}

                    Xray_em = loadHalo(basepath[0], basepath[1], haloID , 0) # erg*cm³/s

                    result[sim][halo_key]['CVI_Line'] = Xray_em['CVI_Line'] # erg*cm³/s
                    result[sim][halo_key]['CV_Line'] = Xray_em['CV_Line'] # erg*cm³/s
                    result[sim][halo_key]['FeXVII_Line'] = Xray_em['FeXVII_Line'] # erg*cm³/s
                    result[sim][halo_key]['NVII_Line'] = Xray_em['NVII_Line'] # erg*cm³/s
                    result[sim][halo_key]['NVI_Line'] = Xray_em['NVI_Line'] # erg*cm³/s
                    result[sim][halo_key]['NeX_Line'] = Xray_em['NeX_Line'] # erg*cm³/s
                    result[sim][halo_key]['OVIII_Line'] = Xray_em['OVIII_Line'] # erg*cm³/s
                    result[sim][halo_key]['OVIIf_Line'] = Xray_em['OVIIf_Line'] # erg*cm³/s
                    result[sim][halo_key]['OVIIr_Line'] = Xray_em['OVIIr_Line'] # erg*cm³/s

    return result

In [11]:
#with open('../data/xray_line_em.pickle', 'wb') as file:
#    pickle.dump(Xray_line_dict(basepaths,basepaths2), file)

In [14]:
def Xray_line_lumdict(sim_dict, sim_dict2):
    '''
    takes 2 dicts as input, sim_dict is { sim: [basepath,snapshot] } where basepath locates the Xray data
    sim_dict2 is the same where basepath2 locates the output data (regular path)
    returns a nested dict { sim: { haloID: { field : value array } } }
    '''
    
    # initialise output dict
    result = {}
    
    #loop for each sim
    for sim2, basepath2 in sim_dict2.items():
        

        # load header
        header = il.groupcat.loadHeader(basepath2[0] , basepath2[1] )
        h = header['HubbleParam']
        a = header['Time']
        BoxSize = header['BoxSize'] *a/h # pkpc
        
        
        # get the ids of relevant mass halos
        group_masses, haloIDs = mass_mask(basepath2[0], basepath2[1]) 
        
        
        # load all halos
        Halos = il.groupcat.loadHalos(basepath2[0], basepath2[1], fields=['GroupPos', 'Group_R_Crit500', 'Group_R_Crit200'])
        
        
        #loop for each sim
        for sim, basepath in sim_dict.items() :
            if sim == sim2:
                
                # start a new dict to save values of each sim
                result[sim] = {}
                
                
                #loop over the halo ids
                for index, haloID in enumerate(haloIDs):

                    halo_key = f'{haloID}'
                    result[sim][halo_key] = {}
                    
                    
                    # define halo data
                    halo_pos = Halos['GroupPos'][haloID] *a/h  # pkpc
                    R200c = Halos['Group_R_Crit200'][haloID] *a/h # pkpc
                    R500c = Halos['Group_R_Crit500'][haloID] *a/h # pkpc

                             
                    # load gas data
                    Gas = il.snapshot.loadHalo(basepath2[0] , basepath2[1] , haloID, 0)            
                    Gas_mass = Gas['Masses'] *10**10/h  # Msun
                    Gas_ea = Gas['ElectronAbundance'] # unitless
                    Gas_density = Gas['Density'] *(10**10/h)/(a/h)**3 # Msun/pkpc³
                    Gas_coords = Gas['Coordinates'] *a/h  # pkpc
                    Centred_gas = shift(Gas_coords, halo_pos, BoxSize)
                    Gas_radius = np.linalg.norm(Centred_gas, axis =1)
                    
                    
                    # load xray data
                    Lines = ['OVIIr_Line', 'OVIIf_Line', 'OVIII_Line', 'NeX_Line', 'NVII_Line', 'FeXVII_Line', 'CVI_Line', 'CV_Line', 'NVI_Line']
                    Xray_line_em = loadHalo(basepath[0], basepath[1], haloID, 0, fields = Lines) # erg*cm³/s
                    
                    
                    # calc Lx
                    Xray_lum = {}
                    nh_conv = 6.768 *10**-32
                    n_h = Gas_density * (0.76/1.6726e-24) * nh_conv  # cm**-3
                    n_e = Gas_ea * n_h # cm**-3
                    Vol = Gas_mass/Gas_density *(3.086*10**21)**3 # cm**3
                    
                    for line in Lines:
                        Xray_lum[line] =  Xray_line_em[line] * n_e * n_h * Vol # erg/s
                    
                    
                    result[sim][halo_key]['CVI_Line'] = Xray_lum['CVI_Line'] # erg*cm³/s
                    result[sim][halo_key]['CV_Line'] = Xray_lum['CV_Line'] # erg*cm³/s
                    result[sim][halo_key]['FeXVII_Line'] = Xray_lum['FeXVII_Line'] # erg*cm³/s
                    result[sim][halo_key]['NVII_Line'] = Xray_lum['NVII_Line'] # erg*cm³/s
                    result[sim][halo_key]['NVI_Line'] = Xray_lum['NVI_Line'] # erg*cm³/s
                    result[sim][halo_key]['NeX_Line'] = Xray_lum['NeX_Line'] # erg*cm³/s
                    result[sim][halo_key]['OVIII_Line'] = Xray_lum['OVIII_Line'] # erg*cm³/s
                    result[sim][halo_key]['OVIIf_Line'] = Xray_lum['OVIIf_Line'] # erg*cm³/s
                    result[sim][halo_key]['OVIIr_Line'] = Xray_lum['OVIIr_Line'] # erg*cm³/s

                    result[sim][halo_key]['radius'] = Gas_radius # pkpc
                    result[sim][halo_key]['R200c'] = R200c # pkpc
                    result[sim][halo_key]['R500c'] = R500c # pkpc
                    
                   
    return result

In [15]:
#with open('../data/xray_line_lum.pickle', 'wb') as file:
#    pickle.dump(Xray_line_lumdict(basepaths, basepaths2), file)
